# Import Libs

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import joblib
from sklearn.preprocessing import LabelEncoder

# Load Dataset

In [32]:
cnc_data = pd.read_csv('predictive_maintenance.csv')

# Show Dataset Head

In [33]:
cnc_data.head(10)

,UDI,Product ID,Product Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No_Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No_Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No_Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No_Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No_Failure
5,6,M14865,M,298.1,308.6,1425,41.9,11,0,No_Failure
6,7,L47186,L,298.1,308.6,1558,42.4,14,0,No_Failure
7,8,L47187,L,298.1,308.6,1527,40.2,16,0,No_Failure
8,9,M14868,M,298.3,308.7,1667,28.6,18,0,No_Failure
9,10,M14869,M,298.5,309.0,1741,28.0,21,0,No_Failure


# Feature Selection

As we can see from the output above, we can see that there are no null values in the dataset, meaning we do not need to remove or interpolate values for missing cases. However, there are some features that we can safely determine will not impact the prediction of failure, including the "UDI," "Product ID," and "Product Type" features. This is because they are only for identification purposes, and do not add any additional information beyond that. Hence, we will drop these features.

In [34]:
cnc_data = cnc_data.drop(columns=["UDI", "Product ID", "Product Type"])

# EDA

In [35]:
print(cnc_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Air temperature [K]      10000 non-null  float64
 1   Process temperature [K]  10000 non-null  float64
 2   Rotational speed [rpm]   10000 non-null  int64  
 3   Torque [Nm]              10000 non-null  float64
 4   Tool wear [min]          10000 non-null  int64  
 5   Target                   10000 non-null  int64  
 6   Failure Type             10000 non-null  object 
dtypes: float64(3), int64(3), object(1)
memory usage: 547.0+ KB
None


# Missing Values Analysis

In [36]:
cnc_data.isnull().sum()

Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Target                     0
Failure Type               0
dtype: int64

# Data Cleaning

In [37]:
cnc_data.columns = [col.strip() for col in cnc_data.columns]

# Dropping Target Column (We will use Feature_Type as the target)

In [38]:
cnc_data = cnc_data.drop(columns=["Target"])

In [39]:
cnc_data.head(10)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type
0,298.1,308.6,1551,42.8,0,No_Failure
1,298.2,308.7,1408,46.3,3,No_Failure
2,298.1,308.5,1498,49.4,5,No_Failure
3,298.2,308.6,1433,39.5,7,No_Failure
4,298.2,308.7,1408,40.0,9,No_Failure
5,298.1,308.6,1425,41.9,11,No_Failure
6,298.1,308.6,1558,42.4,14,No_Failure
7,298.1,308.6,1527,40.2,16,No_Failure
8,298.3,308.7,1667,28.6,18,No_Failure
9,298.5,309.0,1741,28.0,21,No_Failure


In [40]:
cnc_data.columns

Index(['Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]',
       'Failure Type'],
      dtype='object')

# Define features and target

In [41]:
X = cnc_data.drop(columns=['Failure Type'])
y = cnc_data['Failure Type']

In [42]:
X.head(10)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
0,298.1,308.6,1551,42.8,0
1,298.2,308.7,1408,46.3,3
2,298.1,308.5,1498,49.4,5
3,298.2,308.6,1433,39.5,7
4,298.2,308.7,1408,40.0,9
5,298.1,308.6,1425,41.9,11
6,298.1,308.6,1558,42.4,14
7,298.1,308.6,1527,40.2,16
8,298.3,308.7,1667,28.6,18
9,298.5,309.0,1741,28.0,21


In [43]:
y.head(10)

0    No_Failure
1    No_Failure
2    No_Failure
3    No_Failure
4    No_Failure
5    No_Failure
6    No_Failure
7    No_Failure
8    No_Failure
9    No_Failure
Name: Failure Type, dtype: object

# Encode target labels

In [44]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [45]:
# Save the fitted LabelEncoder
joblib.dump(label_encoder, 'label_encoder.joblib')

['label_encoder.joblib']

In [46]:
y_encoded

array([1, 1, 1, ..., 1, 1, 1])

# Split the data

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features

In [48]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, 'scaler.joblib')

['scaler.joblib']

# Data Augmentation using SMOTE

In [49]:
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

In [50]:
X_train_sm.head(10)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
0,298.3,309.1,1616,31.1,195
1,298.2,308.4,1388,53.8,137
2,298.2,307.8,1528,31.1,194
3,300.9,310.8,1599,33.0,7
4,301.4,310.5,1571,33.9,208
5,300.5,310.5,1373,47.4,56
6,297.3,308.8,1469,44.1,150
7,300.8,311.7,1655,32.2,12
8,301.7,310.9,1608,33.0,160
9,298.5,308.2,1654,27.1,13


# Convert labels to categorical

In [20]:
y_train_sm_cat = to_categorical(y_train_sm)
y_test_cat = to_categorical(y_test)

# Model Building

In [21]:
model = Sequential([
    Dense(1024, input_dim=X_train_sm.shape[1], activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(y_train_sm_cat.shape[1], activation='softmax')
])

c:\AkshayTools\Miniconda3\envs\g6_alda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Implement Early Stopping

In [23]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training the Model

In [24]:
model.fit(X_train_sm, y_train_sm_cat, epochs=50, batch_size=64, validation_data=(X_test, y_test_cat), callbacks=[early_stopping])

Epoch 1/50
724/724 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4351 - loss: 3.8397 - val_accuracy: 0.4515 - val_loss: 1.0288
Epoch 2/50
724/724 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7033 - loss: 0.7478 - val_accuracy: 0.5120 - val_loss: 1.0381
Epoch 3/50
724/724 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7638 - loss: 0.6058 - val_accuracy: 0.5175 - val_loss: 1.1327
Epoch 4/50
724/724 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8142 - loss: 0.4698 - val_accuracy: 0.5470 - val_loss: 1.0933
Epoch 5/50
724/724 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8317 - loss: 0.4309 - val_accuracy: 0.6635 - val_loss: 0.7402
Epoch 6/50
724/724 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8362 - loss: 0.4164 - val_accuracy: 0.6925 - val_loss: 0.6881
Epoch 7/50
724/724 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8489 - loss: 0.3801 - val_accuracy: 0.6265 - val_loss: 1.1298
Epoch 8/50
724/724 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8531 - loss: 0.3734 - val_accuracy: 0.

# Evaluate the model

In [25]:
loss, accuracy = model.evaluate(X_test, y_test_cat)
print(f"Test Accuracy: {accuracy:.2f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6912 - loss: 0.6749
Test Accuracy: 0.69


# Save the model

In [27]:
model.save('MLGuard.keras')
print("Model saved to 'MLGuard.keras'")

Model saved to 'MLGuard.keras'


In [62]:
# Sample input data (replace with your actual data)
sample_input = [[298.1, 308.6, 1551, 42.4, 14]]  # Example data

# Convert the sample input to a DataFrame with the same column names
sample_input_df = pd.DataFrame(sample_input, columns=X.columns)

# Standardize the sample input
sample_input_scaled = scaler.transform(sample_input_df)

# Predict the class
predicted_class_encoded = model.predict(sample_input_scaled)
predicted_class = predicted_class_encoded.argmax(axis=1)

# Decode the predicted class
predicted_label = label_encoder.inverse_transform(predicted_class)

print(f"Predicted Failure Type: {predicted_label[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted Failure Type: Random_Failures
